In [47]:
from finance_utils.electrical.china.spot.discrete.probabilistic_forecast import DiscreteSpot, \
    ProbabilisticDiscreteCurve, SpotNoise
from finance_utils.electrical.china.spot.discrete.testback import TestBack
from finance_utils.electrical.china.spot.rule.recycle import SampleRecycle
import numpy
from data_utils.stochastic_utils.distributions.baseclass import ABCDistribution
from data_utils.stochastic_utils.distributions.basic_distributions import NormalDistribution
from data_utils.stochastic_utils.distributions.nonParametricDistribution import HistogramDist, LogHisDist
from data_utils.stochastic_utils.distributions.discrete.basic import DiscreteDistribution
from matplotlib import pyplot
import tqdm
import json

from easy_utils.db_utils.mysql import MySQLLink
from easy_utils.obj_utils.enumerable_utils import flatten
from private.db import mysql_94j6daxc

In [40]:
quantity_min = 0
quantity_max = 30
submitted_min = 0
submitted_max = 30
trigger_rate = 0.05
punishment_rate = 0.5
random_p = 0
random_p_2 = 0.5

In [41]:
db = MySQLLink(
    url=mysql_94j6daxc["url"],
    port=mysql_94j6daxc["port"],
    user=mysql_94j6daxc["user"],
    pwd=mysql_94j6daxc["pwd"],
    db="tianrun_etrade"
)

In [42]:
d,_,__ = db.select(
    "select time_order,dayahead_statement_price,realtime_statement_price,dayahead_clearing_quantity,realtime_clearing_quantity "
    "from spot_real "
    "where station_id='14013006' and date between '2024-5-1' and '2024-8-1'"
    "order by date,time_order"
)
a= numpy.array(d)

d2,_,__ = db.select(
    "select time_order,dayahead_statement_price,realtime_statement_price,dayahead_clearing_quantity,realtime_clearing_quantity "
    "from spot_real "
    "where station_id='14013006' and date between '2024-8-1' and '2024-9-1'"
    "order by date,time_order"
)
a2= numpy.array(d2)

In [43]:
def get_slice(xlist, order, slice):
    return xlist[numpy.where(xlist[:,0]==order)][:,slice]

In [44]:
dp_his_list = []
rp_his_list = []
aq_his_list = []
max_aq = []
for i in range(1,97):
    dp = get_slice(a,i,1)
    rp = get_slice(a,i,2)
    sq = get_slice(a,i,3)
    rq = get_slice(a,i,4)
    aq = sq + rq
    dp_his_list.append(HistogramDist(dp))
    rp_his_list.append(HistogramDist(rp))
    aq_his_list.append(HistogramDist(aq))
    max_aq.append(numpy.max(aq))
    
dp_his_curve = ProbabilisticDiscreteCurve(dp_his_list)
rp_his_curve = ProbabilisticDiscreteCurve(rp_his_list)
aq_his_curve = ProbabilisticDiscreteCurve(aq_his_list, domain_min=0, domain_max=numpy.max(max_aq))

In [45]:
spot = DiscreteSpot(
    dp_his_curve,
    rp_his_curve,
    aq_his_curve
)
print(spot.quantity.value_list)

[5.560247300688938, 5.480086005731936, 5.484021503235705, 5.468258067320753, 5.1329892584482835, 5.493537639071865, 5.076322587329855, 4.910494622763768, 5.229634403226197, 4.595913974479963, 5.152204301098343, 4.7767419332791405, 4.955032261119736, 5.198612891909897, 4.884397845137504, 5.099419356908849, 4.978817204593331, 4.987989254693831, 4.978752691912395, 4.858580646945584, 5.047494630523908, 4.801516126768564, 4.6715699055579405, 4.65037634049949, 4.480817184322624, 4.364591393970674, 4.304709681769855, 4.365473113693216, 4.288967748125395, 4.271333338250397, 4.172946227431297, 4.006677428842874, 3.65380645380738, 3.5161397892275157, 3.4434838688681197, 3.4252365652873955, 3.281129039861823, 2.8793978529360986, 3.538473116700368, 3.099666665433556, 3.1659892462786807, 3.1599247293677384, 2.847129039743895, 2.9805161310472794, 2.953107532014129, 2.934161292060729, 2.9386344175902748, 3.0657204315098383, 2.985053769878162, 3.1228602103571736, 3.0512580636778193, 3.231709682180036,

In [46]:
print(spot.differential_evolution__search(
    submitted_min=0,
    submitted_max=numpy.max(max_aq),
    trigger_rate = trigger_rate,
    punishment_rate = punishment_rate
))

(array([ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   , 39.301, 39.301, 39.301, 39.301,
       39.301,  0.   , 39.301, 39.301,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   , 39.301, 39.301, 39.301, 39.301,  0.   , 39.301, 39.301,
       39.301,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , 39.301,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   , 39.301, 39.301, 39.301,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]), 84340.68194649223)
